<a href="https://colab.research.google.com/github/Jair-RM/Simulacion-I-Figols/blob/main/Ejercicio_21_Oct_(Lineas_de_espera).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
from math import log, exp, sqrt, cos, acos, pi, sin, factorial
from statistics import mean, stdev, variance
from scipy.stats import norm, linregress
import random as rnd
from random import random, gauss
import numpy as np
import matplotlib.pyplot as plt
from time import time
from scipy.optimize import minimize
from numba import jit
# < > ∈ o ∉ | λ
#@jit(nopython=True)

In [ ]:
# Considere una linea de espera con tres servidores en la que los usuarios
# llegan siguiendo un proceso de Poisson con parametro 0.7 usuarios/min. Los
# tiempos de atecion a los usuarios siguen una distribucion exponencial con
# parametros o.5, 0.4 y 0.4 usuarios/min. Ademas, se cumple "primero en llegar,
# primero en salir".
# a) Estime por simulacion el valor esperado del tiempo que
#    permanece desocupado el servidor con parametro 0.5, en el intervalo
#    [0, 100]
# b) Estime por simulacion el valor esperado del tiempo en el que los tres
#    servidores estan simultaneamente vacios en el intervalo [0, 100]
# c) Suponga ahora que solo hay 2 servidores con parametros 0.5 y0.4. Responda
#    los incisos anteriores

In [ ]:
# CHAT GPT

import numpy as np

# Parámetros
arrival_rate = 0.7  # tasa de llegada
service_rates = [0.5, 0.4, 0.4]  # tasas de servicio
simulation_time = 100  # tiempo de simulación
num_simulations = 1000  # número de simulaciones

# Almacenar el tiempo desocupado
total_idle_time = 0

for _ in range(num_simulations):
    current_time = 0
    servers_busy_until = [0, 0, 0]  # tiempo hasta el que cada servidor está ocupado
    idle_time_server_1 = 0

    while current_time < simulation_time:
        # Generar tiempo de llegada
        inter_arrival_time = np.random.exponential(1 / arrival_rate)
        current_time += inter_arrival_time

        if current_time >= simulation_time:
            break

        # Asignar al primer servidor disponible
        for i in range(3):
            if current_time >= servers_busy_until[i]:  # Servidor i está disponible
                service_time = np.random.exponential(1 / service_rates[i])
                servers_busy_until[i] = current_time + service_time
                break

        # Calcular tiempo desocupado del servidor 1
        if current_time < servers_busy_until[0]:
            idle_time_server_1 += servers_busy_until[0] - current_time

    total_idle_time += idle_time_server_1

# Valor esperado del tiempo desocupado
expected_idle_time_server_1 = total_idle_time / num_simulations
print("Tiempo esperado desocupado del servidor 1:", expected_idle_time_server_1)

Tiempo esperado desocupado del servidor 1: 137.39806115404616


In [4]:
from random import expovariate, choice

def n_servidores(l_l, l_s, T):
    infi = 2 * T  # Basta con que sea más grande que el tiempo de simulación
    n = len(l_s)  # Número de servidores
    t = 0
    l = 0  # número de personas en el sistema
    tl = 0
    s_l = list(range(n))  # servidores desocupados en todo momento
    ts = [infi] * n  # tiempos de servicio
    le = 0  # número de personas esperando

    tiempo_desocupado_05 = 0  # Tiempo desocupado del servidor con lambda 0.5
    tiempo_vacios = 0  # Tiempo en que los tres servidores están vacíos
    tiempo_actual_vacio = 0  # Tiempo acumulado en que están vacíos

    while min(tl, min(ts)) < T:  # Mínimo de los próximos tiempos de salida para cada servidor
        t = min(tl, min(ts))

        # Si hay llegada
        if t == tl:
            l += 1
            print("Llegada", "%10.3f" % t, " ", "%4d" % l)
            tl += expovariate(l_l)
            if s_l:  # Servidores vacíos
                servidor = choice(s_l)
                print("   Pasa al servidor:   ", servidor)
                ts[servidor] = t + expovariate(l_s[servidor])
                s_l.remove(servidor)
            else:
                print("   En espera")
                le = le + 1
        else:  # Si hay salida
            l = l - 1
            servidor = ts.index(t)
            print("        ", "%10.3f" % t, "sale por el servidor: ", servidor)
            s_l.append(servidor)

            # Calcular tiempo desocupado del servidor con lambda 0.5
            if servidor == 0:  # Servidor con lambda 0.5
                tiempo_desocupado_05 += t - tiempo_actual_vacio

            if le > 0 and s_l:  # Si hay personas esperando
                print("   Termina espera pasa al servidor: ", servidor)
                ts[servidor] = t + expovariate(l_s[servidor])
                le = le - 1
                s_l.remove(servidor)
            else:
                ts[servidor] = infi  # Sale pero no hay gente esperando

        # Calcular si los tres servidores están vacíos
        if len(s_l) == n:
            tiempo_vacios += t - tiempo_actual_vacio
            tiempo_actual_vacio = t

        print("*******Servidores vacíos ", s_l, le)

    return tiempo_desocupado_05, tiempo_vacios

# Parámetros
l_l = 0.7  # tasa de llegada
l_s = [0.5, 0.4, 0.4]  # tasas de servicio
T = 100  # tiempo de simulación

tiempo_desocupado_05, tiempo_vacios = n_servidores(l_l, l_s, T)

# Calcular el tiempo esperado
esperado_desocupado_05 = tiempo_desocupado_05 / (T / 60)  # convertimos a minutos
esperado_vacios = tiempo_vacios / (T / 60)  # convertimos a minutos

print("\nTiempo esperado que permanece desocupado el servidor 0.5:", esperado_desocupado_05)
print("Tiempo esperado en el que los tres servidores están simultáneamente vacíos:", esperado_vacios)

Llegada      0.000      1
   Pasa al servidor:    1
*******Servidores vacíos  [0, 2] 0
Llegada      2.042      2
   Pasa al servidor:    2
*******Servidores vacíos  [0] 0
Llegada      2.373      3
   Pasa al servidor:    0
*******Servidores vacíos  [] 0
Llegada      2.528      4
   En espera
*******Servidores vacíos  [] 1
Llegada      2.925      5
   En espera
*******Servidores vacíos  [] 2
              3.188 sale por el servidor:  1
   Termina espera pasa al servidor:  1
*******Servidores vacíos  [] 1
Llegada      3.353      5
   En espera
*******Servidores vacíos  [] 2
              3.557 sale por el servidor:  0
   Termina espera pasa al servidor:  0
*******Servidores vacíos  [] 1
              3.781 sale por el servidor:  0
   Termina espera pasa al servidor:  0
*******Servidores vacíos  [] 0
              4.145 sale por el servidor:  1
*******Servidores vacíos  [1] 0
              6.101 sale por el servidor:  0
*******Servidores vacíos  [1, 0] 0
Llegada      7.235      2
   Pasa 

In [2]:
# PROFESOR
from random import expovariate, choice

def n_servidores(l_l, l_s, T):
  suma=0
  infi = 2 * T
  n = len(l_s)
  t = 0
  l = 0
  tl = 0
  s_l = []

  for i in range(n):
    s_l.append(i)
  ts=[]
  for i in range(n):
    ts.append(infi)
  le=0

  while min(tl, min(ts)) < T:
    ta=t
    t = min(tl, min(ts))

    if 0 in s_l:
      suma=suma + t - ta
    if t == tl:
      l += 1
      #print("Llegada", "%10.3f" %t, " ", "%4d" %l)
      tl =tl + expovariate(l_l)

      if len(s_l)>0:
        servidor = choice(s_l)
        #print("   Pasa al servidor:   ", servidor)
        ts[servidor] = t + expovariate(l_s[servidor])
        s_l.remove(servidor)
      else:
        #print("   En espera")
        le =le + 1
    else:
      l=l-1
      servidor = ts.index(t)
      #print("        ", "%10.3f" % t, "sale por el servidor: ", servidor)
      s_l.append(servidor)

      if le>0 and len(s_l)>0:
        #print(   "Termina espera, pasa al servidor:  ", servidor)
        ts[servidor] = t + expovariate(l_s[servidor])
        le=le-1
        s_l.remove(servidor)
      else:
        ts[servidor]=infi
    #print("*******Servidores vacios ", s_l, le)
  return suma

suma=0
nr=100000
for i in range(nr):
  tiempo=n_servidores(0.7, [0.5, 0.4, 0.4], 100)
  suma=suma+tiempo
print(suma/nr)

48.78738691011245
